In [34]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

df_game_events_dummy = pd.read_parquet("Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/transfermarkrt-dados-clean/dummy_data/game_events_dummy.parquet")
df_games = pd.read_parquet('Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/transfermarkrt-dados-clean/games.parquet')
df_club_games = pd.read_parquet("Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/transfermarkrt-dados-clean/club_games.parquet")
df_appereances = pd.read_parquet("Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/transfermarkrt-dados-clean/appearances.parquet")
merged_filtred = pd.read_parquet('Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/clustering/data/merge_filtred_default.parquet')
df_game_events = pd.read_parquet("Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/transfermarkrt-dados-clean/dummy_data/game_events_descr_dummy_filtred.parquet")

In [35]:
game_stats = df_appereances.groupby(['game_id', 'player_club_id'])[['yellow_cards', 'red_cards', 'goals', 'assists']].sum().reset_index()
game_stats.rename(columns={'player_club_id': 'club_id'}, inplace=True)
game_stats

,game_id,club_id,yellow_cards,red_cards,goals,assists
0,2262147,419,0,0,0,0
1,2262147,720,0,0,2,2
2,2262148,338,0,0,1,1
3,2262148,583,1,0,4,3
4,2262149,11,1,0,2,2
...,...,...,...,...,...,...
2875,4331177,418,1,0,2,1
2876,4331178,27,2,0,2,2
2877,4331178,418,2,0,2,2
2878,4343512,16,3,0,0,0


In [36]:
df_games = df_games[['game_id', 'home_club_id', 'away_club_id', 'home_club_name', 'away_club_name', 'home_club_goals', 'away_club_goals', 'home_club_formation', 'away_club_formation']]
df_games

,game_id,home_club_id,away_club_id,home_club_name,away_club_name,home_club_goals,away_club_goals,home_club_formation,away_club_formation
105,2368466,371,5,celtic fc,AC Milan,0,3,4-2-3-1,4-3-2-1
106,2368481,683,58,olympiacos piraeus,RSC Anderlecht,3,1,4-4-1-1,4-1-4-1
111,2384520,141,506,galatasaray,Juventus FC,1,0,3-5-2,3-5-2 flat
212,2490853,610,583,ajax amsterdam,Paris Saint-Germain,1,1,4-3-3 Attacking,4-3-3 Attacking
222,2495311,12,27,as roma,Bayern Munich,1,7,4-3-3 Attacking,3-5-2
...,...,...,...,...,...,...,...,...,...
68274,3651124,2481,418,None,Real Madrid,0,3,4-2-3-1,4-3-3 Attacking
68294,4243700,16,383,borussia dortmund,PSV Eindhoven,2,0,4-2-3-1,4-3-3 Defending
68304,3651114,1082,409,losc lille,None,1,0,4-4-2 double 6,4-3-1-2
68312,3098025,1049,452,valencia cf,None,3,1,4-4-2 double 6,4-4-2 double 6


In [41]:
# Remoção dos times que não possuem nome - Home club
df_games['home_club_name'] = df_games['home_club_name'].replace('None', np.nan)
df_games = df_games[df_games['home_club_name'].notna()]
# Remoção dos times que não possuem nome - Away club
df_games['away_club_name'] = df_games['away_club_name'].replace('None', np.nan)
df_games = df_games[df_games['away_club_name'].notna()]

# Junção dos datasets
Aqui realizamos a junção dos dataset que jugamos relevantes para a analise

Criou uma linha para cada time que em cada uma das partidas de modo que fique correta a disposição dos dados

In [44]:
#Cria um DataFrame para os times da casa
home_teams = df_games[['game_id', 'home_club_id', 'home_club_name', 'home_club_goals', 'home_club_formation', 'away_club_goals']].copy()
home_teams['team_type'] = 'home'
home_teams = home_teams.rename(columns={
    'home_club_id': 'club_id',
    'home_club_name': 'club_name',
    'home_club_goals': 'club_goals',
    'home_club_formation': 'club_formation',
    'away_club_goals': 'suffered_goals'
})

#Cria um DataFrame para os times visitantes
away_teams = df_games[['game_id', 'away_club_id', 'away_club_name', 'away_club_goals', 'away_club_formation', 'home_club_goals']].copy()
away_teams['team_type'] = 'away'
away_teams = away_teams.rename(columns={
    'away_club_id': 'club_id',
    'away_club_name': 'club_name',
    'away_club_goals': 'club_goals',
    'away_club_formation': 'club_formation',
    'home_club_goals': 'suffered_goals'
})

#Transforma os valores de gols sofridos em negativos
home_teams['suffered_goals'] = -home_teams['suffered_goals']
away_teams['suffered_goals'] = -away_teams['suffered_goals']

#Concatena os DataFrames
games_sep_times = pd.concat([home_teams, away_teams])

#Ordena pelo 'game_id'
games_sep_times = games_sep_times.sort_values('game_id')

In [46]:
games_sep_times['club_formation'] = games_sep_times['club_formation'].replace('None', np.nan)
games_sep_times = games_sep_times[games_sep_times['club_formation'].notna()]
games_sep_times

,game_id,club_id,club_name,club_goals,club_formation,suffered_goals,team_type
66271,2367027,15,Bayer 04 Leverkusen,2,4-3-3 Attacking,-4,away
66271,2367027,985,manchester united,4,4-4-1-1,-2,home
37051,2367028,660,Shakhtar Donetsk,2,4-2-3-1,0,away
37051,2367028,681,real sociedad,0,4-3-3 Attacking,-2,home
48243,2367029,418,Real Madrid,6,4-2-3-1,-1,away
...,...,...,...,...,...,...,...
20026,4331177,27,Bayern Munich,1,4-2-3-1,-2,away
8513,4331178,418,Real Madrid,2,4-4-2 double 6,-2,away
8513,4331178,27,bayern munich,2,4-2-3-1,-2,home
51994,4343512,16,borussia dortmund,0,4-3-3 Attacking,-2,home


In [47]:
games_sep_times['club_formation'].value_counts()

club_formation
4-3-3 Attacking    629
4-2-3-1            613
4-4-2 double 6     219
4-1-4-1            123
3-5-2 flat         122
3-4-2-1            100
4-4-2               63
4-3-1-2             61
4-3-3 Defending     55
3-4-3               45
5-4-1               36
3-4-1-2             30
4-4-1-1             29
3-5-2               24
5-3-2               21
4-4-2 Diamond       14
4-1-3-2             10
4-3-2-1              8
3-5-2 Attacking      8
3-1-4-2              7
4-5-1 flat           6
3-4-3 Diamond        3
3-3-3-1              2
4-5-1                2
Name: count, dtype: int64

In [48]:
games_sep_times

,game_id,club_id,club_name,club_goals,club_formation,suffered_goals,team_type
66271,2367027,15,Bayer 04 Leverkusen,2,4-3-3 Attacking,-4,away
66271,2367027,985,manchester united,4,4-4-1-1,-2,home
37051,2367028,660,Shakhtar Donetsk,2,4-2-3-1,0,away
37051,2367028,681,real sociedad,0,4-3-3 Attacking,-2,home
48243,2367029,418,Real Madrid,6,4-2-3-1,-1,away
...,...,...,...,...,...,...,...
20026,4331177,27,Bayern Munich,1,4-2-3-1,-2,away
8513,4331178,418,Real Madrid,2,4-4-2 double 6,-2,away
8513,4331178,27,bayern munich,2,4-2-3-1,-2,home
51994,4343512,16,borussia dortmund,0,4-3-3 Attacking,-2,home


In [49]:
game_stats

,game_id,club_id,yellow_cards,red_cards,goals,assists
0,2262147,419,0,0,0,0
1,2262147,720,0,0,2,2
2,2262148,338,0,0,1,1
3,2262148,583,1,0,4,3
4,2262149,11,1,0,2,2
...,...,...,...,...,...,...
2875,4331177,418,1,0,2,1
2876,4331178,27,2,0,2,2
2877,4331178,418,2,0,2,2
2878,4343512,16,3,0,0,0


In [51]:
merge_game_sep_stats = games_sep_times.merge(game_stats, on=['club_id', 'game_id'])
merge_game_sep_stats_filtred = merge_game_sep_stats[['club_id', 'game_id','club_name', 'club_formation', 'team_type', 'yellow_cards', 'red_cards', 'goals','suffered_goals', 'assists']]

In [55]:
merge_game_sep_stats_filtred['club_name'] = merge_game_sep_stats_filtred['club_name'].str.lower()

C:\Users\Eduar\AppData\Local\Temp\ipykernel_11436\640653327.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_game_sep_stats_filtred['club_name'] = merge_game_sep_stats_filtred['club_name'].str.lower()


In [56]:
stats = merge_game_sep_stats_filtred[['yellow_cards', 'red_cards', 'goals', 'assists']]

cols_to_dummy = merge_game_sep_stats_filtred.drop(columns=['club_id', 'game_id', 'yellow_cards', 'red_cards', 'goals', 'assists'])

cols_dummified = pd.get_dummies(cols_to_dummy)
cols_dummified = cols_dummified.astype(int)

merge_game_sep_stats_filtred_dummy = pd.concat([stats, cols_dummified], axis=1)


In [61]:
from sklearn.preprocessing import MinMaxScaler

# Select the columns to be scaled
columns_to_scale = ['yellow_cards', 'red_cards', 'goals', 'assists']
columns_to_scale_negative = ['suffered_goals']

# Create a MinMaxScaler object
scaler = MinMaxScaler()
scaler_negative = MinMaxScaler(feature_range=(-1, 0))

merge_game_sep_stats_filtred_normalized = merge_game_sep_stats_filtred.copy()

# Apply MinMaxScaler to the selected columns
merge_game_sep_stats_filtred_normalized[columns_to_scale] = scaler.fit_transform(merge_game_sep_stats_filtred[columns_to_scale])
merge_game_sep_stats_filtred_normalized[columns_to_scale_negative] = scaler_negative.fit_transform(merge_game_sep_stats_filtred[columns_to_scale_negative])

# Display the updated DataFrame
merge_game_sep_stats_filtred_normalized

,club_id,game_id,club_name,club_formation,team_type,yellow_cards,red_cards,goals,suffered_goals,assists
0,15,2367027,bayer 04 leverkusen,4-3-3 Attacking,away,0.250,0.0,0.250,-0.500,0.250
1,985,2367027,manchester united,4-4-1-1,home,0.125,0.0,0.500,-0.250,0.500
2,660,2367028,shakhtar donetsk,4-2-3-1,away,0.625,0.0,0.250,0.000,0.250
3,681,2367028,real sociedad,4-3-3 Attacking,home,0.250,0.0,0.000,-0.250,0.000
4,418,2367029,real madrid,4-2-3-1,away,0.125,0.0,0.750,-0.125,0.750
...,...,...,...,...,...,...,...,...,...,...
2218,27,4331177,bayern munich,4-2-3-1,away,0.000,0.0,0.125,-0.250,0.125
2219,418,4331178,real madrid,4-4-2 double 6,away,0.250,0.0,0.250,-0.250,0.250
2220,27,4331178,bayern munich,4-2-3-1,home,0.250,0.0,0.250,-0.250,0.250
2221,16,4343512,borussia dortmund,4-3-3 Attacking,home,0.375,0.0,0.000,-0.250,0.000


In [62]:
stats = merge_game_sep_stats_filtred_normalized[['yellow_cards', 'red_cards', 'goals', 'suffered_goals', 'assists']]

cols_to_dummy = merge_game_sep_stats_filtred_normalized.drop(columns=['club_id', 'game_id','yellow_cards', 'red_cards', 'goals', 'suffered_goals', 'assists'])

cols_dummified = pd.get_dummies(cols_to_dummy)
cols_dummified = cols_dummified.astype(int)

merge_game_sep_stats_filtred_normalized_dummy = pd.concat([stats, cols_dummified], axis=1)

In [63]:
from sklearn.preprocessing import StandardScaler

# Selecionar as colunas a serem padronizadas
columns_to_standardize = ['yellow_cards', 'red_cards', 'goals', 'suffered_goals', 'assists']

# Criar um objeto StandardScaler
scaler = StandardScaler()

# Aplicar a padronização nas colunas selecionadas
merge_game_sep_stats_filtred_standardized = merge_game_sep_stats_filtred.copy()
merge_game_sep_stats_filtred_standardized[columns_to_standardize] = scaler.fit_transform(merge_game_sep_stats_filtred[columns_to_standardize])

# Exibir o DataFrame atualizado
merge_game_sep_stats_filtred_standardized

,club_id,game_id,club_name,club_formation,team_type,yellow_cards,red_cards,goals,suffered_goals,assists
0,15,2367027,bayer 04 leverkusen,4-3-3 Attacking,away,-0.032608,-0.189485,0.449898,-1.842574,0.699477
1,985,2367027,manchester united,4-4-1-1,home,-0.736381,-0.189485,1.970998,-0.374459,2.385047
2,660,2367028,shakhtar donetsk,4-2-3-1,away,2.078709,-0.189485,0.449898,1.093657,0.699477
3,681,2367028,real sociedad,4-3-3 Attacking,home,-0.032608,-0.189485,-1.071202,-0.374459,-0.986092
4,418,2367029,real madrid,4-2-3-1,away,-0.736381,-0.189485,3.492097,0.359599,4.070616
...,...,...,...,...,...,...,...,...,...,...
2218,27,4331177,bayern munich,4-2-3-1,away,-1.440153,-0.189485,-0.310652,-0.374459,-0.143308
2219,418,4331178,real madrid,4-4-2 double 6,away,-0.032608,-0.189485,0.449898,-0.374459,0.699477
2220,27,4331178,bayern munich,4-2-3-1,home,-0.032608,-0.189485,0.449898,-0.374459,0.699477
2221,16,4343512,borussia dortmund,4-3-3 Attacking,home,0.671164,-0.189485,-1.071202,-0.374459,-0.986092


In [64]:
stats = merge_game_sep_stats_filtred_standardized[['yellow_cards', 'red_cards', 'goals', 'assists']]

cols_to_dummy = merge_game_sep_stats_filtred_standardized.drop(columns=['club_id', 'game_id','yellow_cards', 'red_cards', 'goals', 'assists'])

cols_dummified = pd.get_dummies(cols_to_dummy)
cols_dummified = cols_dummified.astype(int)

merge_game_sep_stats_filtred_standardized_dummy = pd.concat([stats, cols_dummified], axis=1)

In [65]:
stats = merge_game_sep_stats_filtred[['yellow_cards', 'red_cards', 'goals','suffered_goals', 'assists']]

cols_to_dummy = merge_game_sep_stats_filtred_normalized.drop(columns=['club_id', 'game_id','yellow_cards', 'red_cards', 'goals', 'suffered_goals', 'assists'])

cols_dummified = pd.get_dummies(cols_to_dummy)
cols_dummified = cols_dummified.astype(int)

merge_game_sep_stats_filtred_dummy = pd.concat([stats, cols_dummified], axis=1)

In [67]:
merge_game_sep_stats_filtred = merge_game_sep_stats[['club_name', 'club_formation', 'team_type', 'yellow_cards', 'red_cards', 'goals','suffered_goals', 'assists']]

# Salvando tabelas na pasta data
1. Merge_Filtred.parquet
2. Merge_Normalized_Dummy.parquet
3. Merge_Standardized_Dummy.parquet

In [68]:
merge_game_sep_stats_filtred.to_parquet("Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/clustering/data/merge_filtred_default.parquet")
merge_game_sep_stats_filtred_dummy.to_parquet("Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/clustering/data/merge_filtred_dummy.parquet")
merge_game_sep_stats_filtred_normalized_dummy.to_parquet("Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/clustering/data/merge_normalized_dummy.parquet")
merge_game_sep_stats_filtred_standardized_dummy.to_parquet("Q:/Universidade/PISI3/pullebyte-analises/DataSet Project/clustering/data/merge_standardized_dummy.parquet")